In [10]:
import hashlib, json, sys, random
random.seed(0)

In [114]:
def hash_me(msg=''):
    if type(msg) != str:
        msg = json.dumps(msg, sort_keys=True)
    return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

In [198]:
print(hash_me('Sharif'))
print(hash_me('Shari'))

524b30f818bf83fc541d0cb25109686e77c39fbed86bf11da3db0130f49e5e15
cac33d7fb72738ea3c59d02292f7be252c03cac466b872d3a16393dbf85dfd80


In [199]:
def make_transaction(max_value=3):
    sign = int(random.getrandbits(1))*2 -1
    amount = random.randint(1, max_value)
    alice_payment = sign * amount
    bob_payment = -1 * alice_payment
    return {'Alice': alice_payment, 'Bob': bob_payment}

In [200]:
make_transaction()

{'Alice': 2, 'Bob': -2}

In [201]:
txs_buffer = [make_transaction() for i in range(30)]

In [202]:
def update_state(txs, state):
    state = state.copy()
    for key in txs:
        if key in state.keys(): state[key] += txs[key]
        else                  : state[key] = txs[key]
    return state

In [203]:
def is_valid_txs(txs, state):
    if sum(txs.values()) != 0: return False
    
    for key in txs.keys():
        if key in state.keys(): acctBalance = state[key]
        else                  : acctBalance = 0
        
        if (acctBalance + txs[key]) < 0: return False
    
    return True

In [204]:
state = {'Alice': 5, 'Bob': 5}

In [205]:
print(is_valid_txs({'Alice': -3, 'Bob': 3},state))  # Basic transaction- this works great!
print(is_valid_txs({'Alice': -4, 'Bob': 3},state))  # But we can't create or destroy tokens!
print(is_valid_txs({'Alice': -6, 'Bob': 6},state))  # We also can't overdraft our account.
print(is_valid_txs({'Alice': -4, 'Bob': 2,'Lisa':2},state)) # Creating new users is valid
print(is_valid_txs({'Alice': -4, 'Bob': 3,'Lisa':2},state)) # But the same rules still apply!

True
False
False
True
False


In [206]:
state = {'Alice':50, 'Bob':50}  # initial state
genesis_block_txns = [state]
genesis_block_contents = {'blockNumber':0,
                          'parentHash':None,
                          'txnCount':1,
                          'txns':genesis_block_txns}
genesis_hash = hash_me(genesis_block_contents)
genesis_block = {'hash':genesis_hash,'contents':genesis_block_contents}
genesis_block_str = json.dumps(genesis_block)

In [207]:
genesisBlockStr

'{"contents": {"blockNumber": 0, "parentHash": null, "txnCount": 1, "txns": [{"Alice": 50, "Bob": 50}]}, "hash": "7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507"}'

In [208]:
chain = [genesis_block]

In [209]:
def make_block(txns, chain):
    parent_block = chain[-1]
    parent_hash = parent_block['hash']
    block_number = parent_block['contents']['blockNumber'] + 1
    block_contents = {'blockNumber': block_number,
                      'parent_hash': parent_hash,
                      'txnCount': len(txns),
                      'txns': txns}
    block_hash = hash_me(block_contents)
    block = {'hash': block_hash,
             'contents': block_contents}
    return block

In [210]:
block_sz_limit = 5

while len(txs_buffer) > 0:
    buffer_start_sz = len(txs_buffer)
    txs_list = []
    while(len(txs_buffer) > 0) & (len(txs_list) < block_sz_limit):
        new_tx = txs_buffer.pop()
        is_valid = is_valid_txs(new_tx, state)
        if is_valid:
            txs_list.append(new_tx)
            state = update_state(new_tx, state)
        else:
            print('Invalid')
            continue
        
    my_block = make_block(txs_list, chain)
    chain.append(my_block)

In [211]:
chain[0]

{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'Alice': 50, 'Bob': 50}]}}

In [212]:
chain[1]

{'hash': '0126826afd9d954667ab75eb42a89b3fa79425aef730959cf653be9d1f9df9e1',
 'contents': {'blockNumber': 1,
  'parent_hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
  'txnCount': 5,
  'txns': [{'Alice': -3, 'Bob': 3},
   {'Alice': -1, 'Bob': 1},
   {'Alice': 3, 'Bob': -3},
   {'Alice': 3, 'Bob': -3},
   {'Alice': 1, 'Bob': -1}]}}

In [213]:
chain[2]

{'hash': '4a2c9d626bce6f30aad64ceb34b5dfec3b9418e2eb38eb9faa01f1bf5e666028',
 'contents': {'blockNumber': 2,
  'parent_hash': '0126826afd9d954667ab75eb42a89b3fa79425aef730959cf653be9d1f9df9e1',
  'txnCount': 5,
  'txns': [{'Alice': -3, 'Bob': 3},
   {'Alice': -1, 'Bob': 1},
   {'Alice': -2, 'Bob': 2},
   {'Alice': -3, 'Bob': 3},
   {'Alice': -1, 'Bob': 1}]}}

In [214]:
chain[3]

{'hash': 'e2f856f2f6532a6cfd565921c41d3b52b038960a4b4c2cd7de9448313a43ddec',
 'contents': {'blockNumber': 3,
  'parent_hash': '4a2c9d626bce6f30aad64ceb34b5dfec3b9418e2eb38eb9faa01f1bf5e666028',
  'txnCount': 5,
  'txns': [{'Alice': 3, 'Bob': -3},
   {'Alice': 1, 'Bob': -1},
   {'Alice': 3, 'Bob': -3},
   {'Alice': -3, 'Bob': 3},
   {'Alice': -2, 'Bob': 2}]}}

In [215]:
state

{'Alice': 49, 'Bob': 51}

In [216]:
def check_block_hash(block):
    expected_hash = hash_me(block['contents'])
    if block['hash'] != expected_hash: raise Exception('wrong hash')
    return True

In [218]:
def check_block_validity(block, parent, state):
    parent_number = parent['contents']['blockNumber']
    parent_hash = parent['hash']
    block_number = block['contents']['blockNumber']
    
    for txn in block['contents']['txns']:
        if isValidTxn(txn, state): state = update_state(txn, state)
        else: raise Exception('invalid txn')
        
    if block_number != parent_number+1: raise Exception('wrong block number')
    
    if block['contents']['parentHash'] != parentHash: raise Exception('wrong hash')
    
    return state

In [ ]:
def check_chain(state):
    if type(chain) == str:
        try:
            chain = json.loads(chain)
            assert(type(chain) == list)
        except: return False
    elif type(chain) != list:
        return False
    
    state = {}

    # for genesis block
    for txn in chain[0]['contents']['txns']:
        state = updateState(txn,state)
    checkBlockHash(chain[0])
    parent = chain[0]
    
    for block in chain[1:]:
        state = checkBlockValidity(block,parent,state)
        parent = block
        
    return state